# SnowEx SWE and Depth
### SWE from LayerData & SWE and Depth from PointData
In this Jupyter Notebook, we save SWE and depth data from the SnowEx database. From the LayerData class, we calculate SWE following the linked resource. We save the different dates when measurements are available. Next, from the PointData class, given the number of available measurements, we save every 1000th record.  

In [1]:
from snowexsql.db import get_db # Import the function to get connect to the db
from snowexsql.conversions import query_to_geopandas # Import a useful function to format that data into a dataframe 
from datetime import date # Import some tools to build dates

from snowexsql.data import PointData # Import our class for the points table
from snowexsql.data import LayerData # Import our LayerData

import matplotlib.pyplot as plt

from geoalchemy2.shape import to_shape
import pandas as pd
import numpy as np

In [2]:
# Connect to the database
db_name = 'snow:hackweek@db.snowexdata.org/snowex' # This is what you will use for all of hackweek to access the db
engine, session = get_db(db_name)

#### LayerData

A resource to calculate SWE: https://snowexsql.readthedocs.io/en/latest/gallery/plot_pit_swe_example.html

In [3]:
q = session.query(LayerData).filter(LayerData.type == 'density')
df = query_to_geopandas(q, engine)

# Convert density to float
df['value'] = df['value'].astype(float)

# Calculate SWE
swe_lambda = lambda row: row['value'] * (row['depth'] - row['bottom_depth']) / 100
df['swe'] = df.apply(swe_lambda, axis=1)

In [4]:
sites = df['site_id'].unique().tolist()

# Loop over data by site and date
swe_values = []
x_values = []
y_values = []
dates_list = []

for site in sites:
    ind1 = df['site_id'] == site
    dates = df['date'][ind1].unique().tolist()

    for date in dates:
        # Grab all density at this site and date
        ind2= df['date'] == date

        profile = df[ind1 & ind2]
        # Check if there is data on this date/site
        if len(profile.index) > 0:
            swe_values.append(profile['swe'].sum())
            x_values.append(profile['geom'].iloc[0].x)
            y_values.append(profile['geom'].iloc[0].y)
            dates_list.append(date)

In [42]:
df_LayerData = pd.DataFrame(list(zip(swe_values, x_values, y_values, dates_list)),
               columns =['swe', 'x', 'y', 'date'])

In [43]:
df_LayerData

,swe,x,y,date
0,254.800000,424863.198248,4.417806e+06,2019-12-19
1,269.000000,424860.677635,4.417811e+06,2020-01-31
2,323.500000,424863.220099,4.417808e+06,2020-02-05
3,407.600000,424864.085795,4.417809e+06,2020-02-12
4,456.200000,424865.806263,4.417811e+06,2020-02-21
...,...,...,...,...
493,320.600000,424960.674960,4.417808e+06,2020-02-12
494,425.400000,424966.571078,4.417800e+06,2020-02-21
495,385.300000,424964.894265,4.417803e+06,2020-02-26
496,342.000000,424964.894265,4.417803e+06,2020-03-04


In [44]:
lat_list = []
long_list = []
for i in range(len(df_LayerData)):
    coord = utm.to_latlon(df_LayerData['x'][i], df_LayerData['y'][i], 12, 'N')
    lat_list.append(coord[0])
    long_list.append(coord[1])

In [46]:
df_LayerData = pd.DataFrame(list(zip(swe_values, lat_list, long_list, dates_list)),
               columns =['swe', 'latitude', 'longitude', 'date'])

In [47]:
df_LayerData

,swe,latitude,longitude,date
0,254.800000,39.90701,-111.87904,2019-12-19
1,269.000000,39.90705,-111.87907,2020-01-31
2,323.500000,39.90703,-111.87904,2020-02-05
3,407.600000,39.90704,-111.87903,2020-02-12
4,456.200000,39.90705,-111.87901,2020-02-21
...,...,...,...,...
493,320.600000,39.90704,-111.87790,2020-02-12
494,425.400000,39.90696,-111.87783,2020-02-21
495,385.300000,39.90699,-111.87785,2020-02-26
496,342.000000,39.90699,-111.87785,2020-03-04


In [48]:
df_LayerData.to_pickle("LayerData_SWE.pkl")

#### PointData
We want to retrieve snow depth and swe (maybe this is GPR swe). Every 1000 points (subselect).

In [ ]:
# Pre-processing (just checking)
qry = "SELECT DISTINCT site_name FROM sites" # Form a typical SQL query and use python to populate the table name
results = engine.execute(qry) # Then we execute the sql command and collect the results
out = ', '.join((row['site_name'] for row in results)) # Create a nice readable string to print the site names using python 
print(out + '\n') # Create a nice readable string to print the site names using python 

# Get the unique datanames in the table
results = session.query(PointData.type).distinct().all()
print('Available types = {}'.format(', '.join([r[0] for r in results])))

# Get the unique instruments in the table
results = session.query(PointData.instrument).distinct().all()
print('Available types = {}'.format(', '.join([str(r[0]) for r in results])))

In [ ]:
from datetime import date

In [ ]:
%%time

full_values = []
full_dates = []
full_instruments = []
full_latitudes = []
full_longitudes = []
full_elevations = []
full_utm_zones = []

for i in range(len(np.unique(dates_list))):
    # Pick a dataset
    dataset = 'depth' #depth

    # Site name
    site_name = "Grand Mesa"

    # Pick a date
    collection_date = date(dates_list[i].year, dates_list[i].month, dates_list[i].day)

    # Get a session
    engine, session = get_db(db_name)

    qry = session.query(PointData)
    qry = qry.filter(PointData.type == dataset)
    qry = qry.filter(PointData.date == collection_date)

    # Execute the query and convert to geopandas in one handy function
    df = query_to_geopandas(qry, engine)
    print(df)
    # print(list(df.columns))
    len_df = np.arange(len(df))
    df2 = df[(len_df % 1000 == 1)]
    
    full_values.append(list(df2['value']))
    full_dates.append(list(df2['date']))
    full_instruments.append(list(df2['instrument']))
    full_latitudes.append(list(df2['latitude']))
    full_longitudes.append(list(df2['longitude']))
    full_elevations.append(list(df2['elevation']))
    full_utm_zones.append(list(df2['utm_zone']))
    # how many did we retrieve?
    print(i, f'{len(df2.index)} records')
session.close()

In [ ]:
# we flatten our list of lists
flat_full_values = [x for xs in full_values for x in xs]; print(len(flat_full_values))
flat_full_dates = [x for xs in full_dates for x in xs]; print(len(flat_full_dates))
flat_full_instruments = [x for xs in full_instruments for x in xs]; print(len(flat_full_instruments))
flat_full_latitudes = [x for xs in full_latitudes for x in xs]; print(len(flat_full_latitudes))
flat_full_longitudes = [x for xs in full_longitudes for x in xs]; print(len(flat_full_longitudes))
flat_full_elevations = [x for xs in full_elevations for x in xs]; print(len(flat_full_elevations))

In [ ]:
#change depth to SWE if saving SWE
df_PointData = pd.DataFrame(list(zip(flat_full_values, flat_full_dates, flat_full_instruments, 
                                     flat_full_latitudes, flat_full_longitudes, flat_full_elevations)),
               columns =['depth', 'date', 'instrument', 'latitude', 'longitude', 'elevation']) 

In [ ]:
df_PointData.head()

In [ ]:
# saving our dataframe so that we do not need to load it everytime
# df_PointData.to_pickle("PointData_SWE.pkl")
df_PointData.to_pickle("PointData_depth.pkl")

#### Matching LayerData and PointData

In [3]:
import utm

In [19]:
PointData_SWE = pd.read_pickle("PointData_SWE.pkl")
print(PointData_SWE)

         SWE        date         instrument   latitude   longitude  elevation
0      229.0  2020-01-31   Mala 800 MHz GPR  39.024950 -108.169552    3106.40
1      240.0  2020-01-31   Mala 800 MHz GPR  39.024962 -108.169678    3103.62
2      275.0  2020-01-31   Mala 800 MHz GPR  39.025108 -108.169702    3101.36
3      326.0  2020-01-31   Mala 800 MHz GPR  39.024778 -108.169581    3105.89
4      291.0  2020-01-31   Mala 800 MHz GPR  39.025262 -108.169362    3106.17
...      ...         ...                ...        ...         ...        ...
16141  203.0  2020-02-04  Mala 1600 MHz GPR  39.031349 -108.139465    3147.60
16142  216.0  2020-02-04  Mala 1600 MHz GPR  39.030843 -108.139429    3143.02
16143  161.0  2020-02-04  Mala 1600 MHz GPR  39.031217 -108.139108    3141.90
16144  237.0  2020-02-04  Mala 1600 MHz GPR  39.031177 -108.139100    3141.01
16145  273.0  2020-02-04  Mala 1600 MHz GPR  39.031174 -108.138983    3141.70

[16146 rows x 6 columns]


In [20]:
PointData_depth = pd.read_pickle("PointData_depth.pkl")
PointData_depth

,depth,date,instrument,latitude,longitude,elevation
0,84.0,2020-01-31,Mala 800 MHz GPR,39.024950,-108.169552,3106.40
1,88.0,2020-01-31,Mala 800 MHz GPR,39.024962,-108.169678,3103.62
2,101.0,2020-01-31,Mala 800 MHz GPR,39.025108,-108.169702,3101.36
3,119.0,2020-01-31,Mala 800 MHz GPR,39.024778,-108.169581,3105.89
4,107.0,2020-01-31,Mala 800 MHz GPR,39.025262,-108.169362,3106.17
...,...,...,...,...,...,...
16403,87.0,2020-02-04,magnaprobe,39.016670,-108.163280,3103.20
16404,99.0,2020-02-04,mesa,39.030900,-108.141340,3136.08
16405,116.0,2020-02-04,magnaprobe,39.032300,-108.166700,3106.70
16406,92.0,2020-02-12,None,38.959240,-106.990630,NaN


In [45]:
df_LayerData # note that these x and y are in UTM

,swe,x,y,date
0,254.800000,424863.198248,4.417806e+06,2019-12-19
1,269.000000,424860.677635,4.417811e+06,2020-01-31
2,323.500000,424863.220099,4.417808e+06,2020-02-05
3,407.600000,424864.085795,4.417809e+06,2020-02-12
4,456.200000,424865.806263,4.417811e+06,2020-02-21
...,...,...,...,...
493,320.600000,424960.674960,4.417808e+06,2020-02-12
494,425.400000,424966.571078,4.417800e+06,2020-02-21
495,385.300000,424964.894265,4.417803e+06,2020-02-26
496,342.000000,424964.894265,4.417803e+06,2020-03-04
